In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
def shrink_df(df):
    df.user_id = df.user_id.astype(np.uint16)
    df.movie_id = df.movie_id.astype(np.uint16)
    df.rating = df.rating.astype(np.uint8)
    
    return df

In [ ]:
df = pd.read_csv("data/user_data/ml_100k_user_data.csv")

In [ ]:
y = df.rating
X = df.drop(["rating"], axis = 1)

In [ ]:
del df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [ ]:
X_train['rating'] = y_train
X_test['rating'] = y_test

In [ ]:
del y_train
del y_test

In [ ]:
#X_train.to_csv("data/new_pp/udata_train_100k.csv", index=False)

In [ ]:
#X_test.to_csv("data/new_pp/udata_test_100k.csv", index=False)

In [ ]:
# read from the split csv

In [3]:
df_train = pd.read_csv("data/new_pp/udata_train_100k.csv")

In [4]:
df_test = pd.read_csv("data/new_pp/udata_test_100k.csv")

In [5]:
one_hot_train = pd.get_dummies(df_train.movie_id, prefix="movie_id", dtype=np.uint8)

In [ ]:
one_hot_train["user_id"] = df_train.user_id

In [ ]:
# reorder and sort the columns, write to csv, delete variables
one_hot_train = one_hot_train[one_hot_train.columns.tolist()[-1:] + one_hot_train.columns.tolist()[:-1]]

In [ ]:
group_train = one_hot_train.groupby("user_id")

In [ ]:
exposure_train = pd.DataFrame(columns=one_hot_train.columns)

In [ ]:
for i in tqdm(one_hot_train.user_id.unique().tolist()):
    temp = pd.DataFrame(group_train.get_group(i).drop("user_id", axis=1).sum()).T
    temp["user_id"] = i
    temp = temp[temp.columns.tolist()[-1:] + temp.columns.tolist()[:-1]]
    exposure_train = pd.concat([exposure_train, temp], ignore_index=True)

In [ ]:
exposure_train.shape

In [ ]:
# exposure_train.to_csv("data/new_pp/exposure_100k_train.csv", index=False)

In [ ]:
train_df = pd.read_csv("data/new_pp/exposure_100k_train.csv")

In [ ]:
train_df.shape

In [ ]:
train_cols = train_df.columns.tolist()[1:]

In [ ]:
del train_df

In [ ]:
one_hot_test = pd.get_dummies(df_test.movie_id, prefix="movie_id", dtype=np.uint8)

In [ ]:
one_hot_test["user_id"] = df_test.user_id

In [ ]:
del df_test

In [ ]:
one_hot_test.shape

In [ ]:
one_hot_test.head()

In [ ]:
train_cols = set(train_cols)
test_cols = set(one_hot_test.columns[:-1])

In [ ]:
col_to_drop = list(test_cols - train_cols.intersection(test_cols))

In [ ]:
col_to_drop[0]

In [ ]:
one_hot_test = one_hot_test.drop(col_to_drop, axis=1)

In [ ]:
one_hot_test.shape

In [ ]:
one_hot_test.head()

In [ ]:
col_to_add = list(train_cols - train_cols.intersection(test_cols))

In [ ]:
col_to_add[0]

In [ ]:
one_hot_test.shape

In [ ]:
data = np.zeros((one_hot_test.shape[0],len(col_to_add)), dtype=np.uint8)

temp_df = pd.DataFrame(data, columns=col_to_add)

In [ ]:
temp_df.head()

In [ ]:
one_hot_test_n = pd.concat([one_hot_test, temp_df], axis=1)

In [ ]:
one_hot_test_n.shape

In [ ]:
user_col = one_hot_test_n.user_id

In [ ]:
one_hot_test_n = one_hot_test_n.drop(["user_id"], axis=1)

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


In [ ]:
col = one_hot_test_n.columns.tolist()
col.sort(key=natural_keys)

In [ ]:
one_hot_test_n["user_id"] = user_col

In [ ]:
one_hot_test_n = one_hot_test_n[one_hot_test_n.columns.tolist()[-1:] + one_hot_test_n.columns.tolist()[:-1]]

In [ ]:
one_hot_test_n.shape

In [ ]:
one_hot_test_n.head()

In [ ]:
grouped = one_hot_test_n.groupby("user_id")

In [ ]:
exposure_test = pd.DataFrame(columns=one_hot_test_n.columns)

In [ ]:
for i in tqdm(one_hot_test_n.user_id.unique().tolist()):
    temp = pd.DataFrame(grouped.get_group(i).drop("user_id", axis=1).sum()).T
    temp["user_id"] = i
    temp = temp[temp.columns.tolist()[-1:] + temp.columns.tolist()[:-1]]
    exposure_test = pd.concat([exposure_test, temp], ignore_index=True)

In [ ]:
exposure_test.shape

In [ ]:
exposure_test.head()

In [ ]:
exposure_test.to_csv("data/new_pp/exposure_100k_test.csv", index=False)

In [ ]:
for col in tqdm(one_hot.columns.to_list()[:-1]):
    one_hot[col] = np.multiply(one_hot.rating, one_hot[col])

In [ ]:
one_hot_train["user_id"] = df_train.user_id